In [167]:
import pandas as pd
import numpy as np

In [171]:
import requests
import json

In [172]:
def callApi(method, stock,isEarnings = False ):
    key = "token=" + "pk_ee5af08dfb9a419aaba2cbee77b80165"
    host = "https://cloud.iexapis.com/stable/"
    ticker = "stock/"+ stock
    api_method ="/" + method 
    query = "?"
    add = "&"
    last = "/4"
    period = "period=annual"
    if isEarnings:
        url = host + ticker + api_method + last + query + period + add + key 
    else:
        url = host + ticker + api_method + query + key 
    try:
        response = requests.request("GET", url)
        balance_sheet = json.loads(response.text)
        return balance_sheet
    except ValueError:
        print("Unknown expeption on" + stock)
        return {}

In [204]:
callApi("quote", "AAPL")

{'symbol': 'AAPL',
 'companyName': 'Apple, Inc.',
 'primaryExchange': 'NASDAQ',
 'calculationPrice': 'close',
 'open': 120,
 'openTime': 1599226202030,
 'openSource': 'official',
 'close': 120.96,
 'closeTime': 1599249600801,
 'closeSource': 'official',
 'high': 123.7,
 'highTime': 1599263998145,
 'highSource': '15 minute delayed price',
 'low': 110.89,
 'lowTime': 1599230668001,
 'lowSource': '15 minute delayed price',
 'latestPrice': 120.96,
 'latestSource': 'Close',
 'latestTime': 'September 4, 2020',
 'latestUpdate': 1599249600801,
 'latestVolume': 332607163,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 120,
 'delayedPriceTime': 1599263998145,
 'oddLotDelayedPrice': 453.263,
 'oddLotDelayedPriceTime': 1599252382500,
 'extendedPrice': 120.01,
 'extendedChange': -0.95,
 'extendedChangePercent': -0.00785,
 'extendedPriceTime': 1599263998144,
 'previousClose': 120.88,
 'previousVolume': 257599640,
 'change': 0.08,
 'changePercent': 0.00

In [173]:
def peRatio(quote):
    if not("peRatio" in quote.keys()) and isinstance(float(quote["peRatio"]), float):
        return 0.0
    return float(quote["peRatio"])

In [174]:
def priceToBook(stats):
    if not("priceToBook" in stats.keys()) and isinstance(float(stats["priceToBook"]), float):
        return 0.0
    return float(stats["priceToBook"])

In [175]:
def debtToEquity(stats):
    if not("debtToEquity" in stats.keys())and stats["debtToEquity"] is None:
        return 0.0
    return float(stats["debtToEquity"])

In [176]:
def pegRatio(stats):
    if not("pegRatio" in stats.keys()):
        return 0.0
    if stats["pegRatio"] is not None:
        return float(stats["pegRatio"])
    else:
        return 0.0

In [177]:
# Current Ratio= Current Assets / Current Liabilities
def currentRatio(balance_sheet):
    if not("balancesheet" in balance_sheet.keys()) or len(balance_sheet["balancesheet"]) == 0:
        return 0.0
    latest_balance_sheet = balance_sheet["balancesheet"][0]
    if not("totalAssets" in latest_balance_sheet.keys()) and not("totalLiabilities" in latest_balance_sheet.keys()):
        return 0.0
    return float(latest_balance_sheet["totalAssets"]/ latest_balance_sheet["totalLiabilities"])
    

In [178]:
def checkErningProgression(earnings):
    if not("earnings" in earnings.keys()) or len(earnings["earnings"]) == 0:
        return False
    earningsSet = earnings["earnings"][:-1]
    yearAgoChangePercent = np.array([])
    for earning in earningsSet:
        if not("yearAgoChangePercent" in earning.keys()):
            return False
        yearAgoChangePercent = np.append(yearAgoChangePercent, earning["yearAgoChangePercent"])
    isDecending = (yearAgoChangePercent == np.sort(yearAgoChangePercent)[::-1]).all()
    return isDecending

In [179]:
def avgEPS(earnings):
    # Average Ernings 
    if not("earnings" in earnings.keys()) or len(earnings["earnings"]) == 0:
        return 0.0
    earningsSet = earnings["earnings"]
    count = 0
    epsSum = 0
    for earning in earningsSet:
        if "actualEPS" in earning.keys():
            epsSum += earning["actualEPS"] 
            count += 1
    return float(epsSum/count)

In [202]:
# COMPOUND ANNUAL GROWTH RATES (CAGR)
def CAGR(earnings):
    if not("earnings" in earnings.keys()) or len(earnings["earnings"]) == 0:
        return 0.0
    earningsSet = earnings["earnings"]
    periods = 0
    eps = np.array([])
    for earning in earningsSet:      
        if "actualEPS" in earning.keys():
            periods += 1
            eps = np.append(eps, earning["actualEPS"]) 
    if periods > 1 :
        first = eps[-1]
        last = eps[0]                  
        return (last/first)**(1/periods)-1
    else:
        return 0

In [185]:
def intrinsicValueBenGraham(earnings):
    #Intrinsic value = [EPS × (7 + 1 *g) × 4.4]/Y 
    Bond_Yeild_AAA = 1.51
    PE_Ratio = 7
    Growth_Aggressive_Value = 1.125
    eps = avgEPS(earnings)
    cgrs = CAGR(earnings)
    intrinsicValue = eps * (PE_Ratio + Growth_Aggressive_Value *cgrs) * 4.4 / Bond_Yeild_AAA 
    print(intrinsicValue)
    return intrinsicValue

In [205]:
tickers = ['DAL', 'BA', 'JPM', 'MRNA', 'HTZ', 'LUV','TELA', 'FB', 'MAR']
dataSet = [] 
for ticker in tickers:
    data = {}
    points = 0
    data["Ticker"] = ticker
    # Major data set 
    quote = callApi("quote", ticker)
    stats = callApi("advanced-stats", ticker)
    balance_sheet = callApi("balance-sheet", ticker)
    earnings = callApi("earnings", ticker, True)
    
    data["Price"] = quote["close"]
    # PE Ratio < 15
    pe_ratio = peRatio(quote)
    if pe_ratio != 0: 
        data["PE Ratio < 15"] = pe_ratio
    else: 
        data["PE Ratio < 15"] = "NA"

    # Price To Book Ratio  
    price_to_book = priceToBook(stats)
    if price_to_book != 0: 
        data["Price To Book < 1.5"] = price_to_book
    else: 
        data["Price To Book < 1.5"] = "NA"
    
    if pe_ratio * price_to_book < 22.5 and pe_ratio * price_to_book > 0: 
        points += 1
    
    # PEG Ratio 
    peg_ratio = pegRatio(stats)
    if peg_ratio != 0: 
        data["PEG Ratio(1 and 0)"] = peg_ratio
    else: 
        data["PEG Ratio(1 and 0)"] = "NA"
    if peg_ratio < 1.1 and peg_ratio > 0:
        points += 1
    
    # Debt To Equity Ratio 
    # Warent buffet say < 0.5
    debt_Equity = debtToEquity(stats)
    if debt_Equity != 0: 
        data["Debt to Equity (0 and 1.5)"] = debt_Equity
    else: 
        data["Debt to Equity (0 and 1.5)"] = "NA"
    if debt_Equity < 1.6 and debt_Equity > 0:
        points += 1
        
    # Current Ratio 
    current_ratio = currentRatio(balance_sheet)
    if current_ratio != 0: 
        data["Current Ratio > 1.5"] = current_ratio
    else: 
        data["Current Ratio > 1.5"] = "NA"
        
    if current_ratio > 1.4:
        points += 1
    
    # Earning Progression Last 4 year 
    isProgressing = checkErningProgression(earnings)
    data["Earning Progression"] =  isProgressing
    if isProgressing:
        points += 1
    
    data["Points(5)"] = points
    
    #Intrinsic value  
    data["Intrinsic value"] = intrinsicValueBenGraham(earnings)
    dataSet.append(data.copy())
pd.DataFrame(dataSet)    

119.92838451514119


/Users/Prajeesh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


nan
171.10980175335771
-29.747671505346972
1.7710651577900811
80.79175148263289
-348.794701986755
126.57897421570856
105.34033113121612


,Ticker,Price,PE Ratio < 15,Price To Book < 1.5,PEG Ratio(1 and 0),Debt to Equity (0 and 1.5),Current Ratio > 1.5,Earning Progression,Points(5),Intrinsic value
0,DAL,31.77,-5.46,1.319489,0.02,1.12,1.136698,True,3,119.928385
1,BA,171.05,-33.97,-11.204480,-1.92,-3.31,0.934682,False,0,NaN
2,JPM,103.52,13.92,1.207238,-0.27,2.08,1.089691,False,1,171.109802
3,MRNA,62.60,-44.63,21.025652,-1.83,0.12,6.466092,True,3,-29.747672
4,HTZ,1.44,-0.18,0.127155,NA,10.70,1.029987,True,1,1.771065
5,LUV,39.39,96.62,2.363208,-0.33,0.41,1.440084,False,2,80.791751
6,TELA,13.19,-5.48,6.140537,0.43,0.98,2.856766,True,4,-348.794702
7,FB,282.73,34.29,6.726721,0.35,0.11,4.776741,False,3,126.578974
8,MAR,104.82,74.81,48.356612,-0.53,17.00,0.996933,False,0,105.340331
